In [27]:
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd
import numpy as np

In [28]:
os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Ilostat")

In [29]:
df = pd.read_csv("ilostat_paises.csv")

In [4]:
df.head(10)

,Reference area,Selecionado,Source,Sex,Time,Total,Public,Private,Not elsewhere classified,Unnamed: 9
0,Afghanistan,NaN,HIES - Living Conditions Survey,Total,2012,6420.0,680.0,5740.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
1,Afghanistan,NaN,HIES - Living Conditions Survey,Male,2012,5358.0,589.0,4769.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
2,Afghanistan,NaN,HIES - Living Conditions Survey,Female,2012,1061.0,91.0,971.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
3,Afghanistan,NaN,HIES - Living Conditions Survey,Total,2014,7003.0,722.0,6281.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
4,Afghanistan,NaN,HIES - Living Conditions Survey,Male,2014,5319.0,649.0,4670.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
5,Afghanistan,NaN,HIES - Living Conditions Survey,Female,2014,1684.0,73.0,1611.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
6,Afghanistan,NaN,HIES - Living Conditions Survey,Total,2017,6397.0,649.0,5748.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
7,Afghanistan,NaN,HIES - Living Conditions Survey,Male,2017,5018.0,584.0,4435.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
8,Afghanistan,NaN,HIES - Living Conditions Survey,Female,2017,1378.0,65.0,1313.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...
9,Afghanistan,NaN,LFS - Labour Force Survey,Total,2020,6079.0,532.0,5547.0,NaN,Notes\n\nEmployment (thousands) \n-Repository:...


In [7]:
df.columns

Index(['Reference area', 'Selecionado', 'Source', 'Sex', 'Time', 'Total',
       'Public', 'Private', 'Not elsewhere classified', 'Unnamed: 9'],
      dtype='object')

In [30]:
## Passar para long

df_long = df.melt(id_vars =['Reference area', 'Source','Time','Sex'], 
        value_vars =['Public', 'Private'], 
        var_name ='Setor', 
        value_name ='Total')
df_long

,Reference area,Source,Time,Sex,Setor,Total
0,Afghanistan,HIES - Living Conditions Survey,2012,Total,Public,680.0
1,Afghanistan,HIES - Living Conditions Survey,2012,Male,Public,589.0
2,Afghanistan,HIES - Living Conditions Survey,2012,Female,Public,91.0
3,Afghanistan,HIES - Living Conditions Survey,2014,Total,Public,722.0
4,Afghanistan,HIES - Living Conditions Survey,2014,Male,Public,649.0
...,...,...,...,...,...,...
6347,Zimbabwe,LFS - Labour Force Survey,2014,Male,Private,2888.0
6348,Zimbabwe,LFS - Labour Force Survey,2014,Female,Private,3034.0
6349,Zimbabwe,LFS - Labour Force Survey,2019,Total,Private,4587.0
6350,Zimbabwe,LFS - Labour Force Survey,2019,Male,Private,2271.0


In [31]:
## fazer o total

df = df_long[df_long['Sex']=="Total"]


In [33]:
df

,Reference area,Source,Time,Sex,Setor,Total
0,Afghanistan,HIES - Living Conditions Survey,2012,Total,Public,680.0
3,Afghanistan,HIES - Living Conditions Survey,2014,Total,Public,722.0
6,Afghanistan,HIES - Living Conditions Survey,2017,Total,Public,649.0
9,Afghanistan,LFS - Labour Force Survey,2020,Total,Public,532.0
12,Afghanistan,LFS - Labour Force Survey,2021,Total,Public,643.0
...,...,...,...,...,...,...
6337,Zambia,LFS - Labour Force Survey,2019,Total,Private,5101.0
6340,Zambia,LFS - Labour Force Survey,2020,Total,Private,5249.0
6343,Zimbabwe,LFS - Labour Force Survey,2011,Total,Private,5057.0
6346,Zimbabwe,LFS - Labour Force Survey,2014,Total,Private,5922.0


In [8]:
df.columns

Index(['Reference area', 'Source', 'Time', 'Sex', 'Setor', 'Total'], dtype='object')

In [34]:
x=df[['Reference area', 'Source', 'Time', 'Setor', 'Total']].pivot_table(index=['Reference area','Time' ], values='Total',aggfunc=np.sum)
y = pd.DataFrame(x)
total = y.reset_index()

In [35]:
df= df[['Reference area', 'Source', 'Time', 'Setor', 'Total']]
df= df.rename(columns={"Total":"quantidade_vinculos"})

In [36]:
df1= df.merge(total,right_on=['Reference area','Time'],left_on=['Reference area','Time'])

In [37]:
df1['prop'] = df1['quantidade_vinculos']/df1['Total']

In [54]:
df1['prop'] = np.where(df1['prop']==1,np.nan,df1['prop'])

In [88]:
df1[df1['pais']=="Brazil"]

,pais,fonte_pesquisa,ano,setor,quantidade_vinculos,total_ano,prop
284,Brazil,HS - Pesquisa Nacional por Amostra de Domicílios,2011,Public,10959.0,90931.0,0.120520
285,Brazil,HS - Pesquisa Nacional por Amostra de Domicílios,2011,Private,79972.0,90931.0,0.879480
286,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2012,Public,10954.0,89317.0,0.122642
287,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2012,Private,78363.0,89317.0,0.877358
288,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2013,Public,10942.0,90770.0,0.120546
289,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2013,Private,79828.0,90770.0,0.879454
290,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2014,Public,11179.0,91977.0,0.121541
291,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2014,Private,80798.0,91977.0,0.878459
292,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2015,Public,11124.0,91962.0,0.120963
293,Brazil,HS - Pesquisa Nacional por Amostra de Domicíli...,2015,Private,80838.0,91962.0,0.879037


In [ ]:
## Tratamento adicional:

#Quando a prop for 1.0 ignorar

In [55]:
paises = ["Japan","United Kingdom","Canada","Australia","Germany"]

In [56]:
selecionados = df1[df1['Reference area'].isin(paises)]

In [58]:
selecionados.to_csv("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\DADOS\\415 - Repositório de Dados\\Bases solicitadas externamente\\paises_selecionados.csv", sep=";",encoding="latin1", decimal=",")

In [73]:
df1= df1.rename(columns={'Time':'ano','Reference area':'pais','Source':'fonte_pesquisa','Total':'total_ano','Setor':'setor'})

In [74]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2172 entries, 0 to 2171
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pais                 2172 non-null   object 
 1   fonte_pesquisa       2172 non-null   object 
 2   ano                  2172 non-null   int64  
 3   setor                2172 non-null   object 
 4   quantidade_vinculos  2130 non-null   float64
 5   total_ano            2172 non-null   float64
 6   prop                 2124 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 135.8+ KB


In [71]:
## Subindo para GBQ

client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao') # verificar a categoria do dado

In [80]:
min(df1['ano'])

2010

In [76]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
bigquery.SchemaField('ano','Integer',description='Ano de coleta da informação'),
bigquery.SchemaField('setor','STRING',description='se o setor é público ou privado'),
bigquery.SchemaField('quantidade_vinculos','float',description='total de pessoas que trabalham naquele setor'),
bigquery.SchemaField('total_ano','float',description='total de pessoas que trabalham naquele ano'),
 bigquery.SchemaField('prop','Float',description='Proporção de vínculos naquele setor'),
 
 ]

In [78]:
table_ref = dataset_ref.table('ILOSTAT_todos_paises') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=8c7a8c46-4d38-4425-b3f5-7b4514fe06a8>

In [81]:
table = client.get_table('perfil_remuneracao.ILOSTAT_todos_paises') 

In [87]:
table.schema


[SchemaField('pais', 'STRING', 'NULLABLE', 'Nome do país selecionado', (), None),
 SchemaField('fonte_pesquisa', 'STRING', 'NULLABLE', 'De qual pesquisa nacional foi extraído aquele dado.', (), None),
 SchemaField('ano', 'INTEGER', 'NULLABLE', 'Ano de coleta da informação', (), None),
 SchemaField('setor', 'STRING', 'NULLABLE', 'se o setor é público ou privado', (), None),
 SchemaField('quantidade_vinculos', 'FLOAT', 'NULLABLE', 'total de pessoas que trabalham naquele setor', (), None),
 SchemaField('total_ano', 'FLOAT', 'NULLABLE', 'total de pessoas que trabalham naquele ano', (), None),
 SchemaField('prop', 'FLOAT', 'NULLABLE', 'Proporção de vínculos naquele setor', (), None)]